In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv
from langchain_groq import ChatGroq
# from langchain_openai import ChatOpenAI
import os

In [2]:
load_dotenv()
# print("Loaded key:", os.getenv("GROQ_API_KEY"))

True

In [3]:
# Import the API Key for LLM
# os.environ.pop("OPENAI_API_KEY", None)

In [4]:
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),  # Explicitly pass
    model_name="llama3-8b-8192",
    temperature=0.1,
    streaming=False,
)

In [5]:
# Create state type
class LLMState(TypedDict):
    question: str
    answer: str

In [6]:
def llm_qa(state: LLMState) -> LLMState:
    question = state['question']
    prompt = f"Answer the following question: {question}"
    answer = llm.invoke(prompt).content
    state['answer'] = answer
    return state

In [7]:
# Build the graph
graph = StateGraph(LLMState)
graph.add_node('llm_qa', llm_qa)
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

In [8]:
# Compile and execute
workflow = graph.compile()
initial_state = {'question': 'How far is the moon from the Earth?'}
final_state = workflow.invoke(initial_state)
# Compile
print(final_state) # final_state['answer'] to see only answer

{'question': 'How far is the moon from the Earth?', 'answer': 'The average distance from the Earth to the Moon is about 384,400 kilometers (238,900 miles). This is called the "lunar distance" or "lunar mean distance." However, the Moon\'s orbit is not a perfect circle and its distance from Earth varies slightly over the course of a month, due to the elliptical shape of its orbit.\n\nAt its closest point, called "perigee," the Moon is about 356,400 kilometers (221,500 miles) away from Earth. At its farthest point, called "apogee," the Moon is about 405,500 kilometers (252,000 miles) away from Earth.\n\nSo, to summarize:\n\n* Average distance: 384,400 kilometers (238,900 miles)\n* Closest point (perigee): 356,400 kilometers (221,500 miles)\n* Farthest point (apogee): 405,500 kilometers (252,000 miles)'}


In [ ]:
# We can get the answer by invoking llm only
# llm.invoke('How far is the moon from the Earth?').content
# but our end goal is different


In [ ]:
##  Simple workflow 